# Imports, load data

In [1]:
import pandas as pd
import numpy as np
from scipy.special import logsumexp
from tqdm import tqdm
from ipywidgets import IntProgress
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from src.RBM import RBM, DRBM
from src.utils import one_hot_encode
from src.data import DataLoader


# data = pd.read_csv('reduced_data.csv').set_index('CEPH ID')

data_loader = DataLoader()
num_data = data_loader.num_data
num_dims = data_loader.num_dims
num_targets = data_loader.num_targets

# data = data.replace(to_replace=2, value=1)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# X = data.values[:, :1000].astype(np.int8)
# y = data.values[:, -2]

# Y, y_keys = one_hot_encode(y)

# # to_delete = np.argwhere(np.isclose(X.mean(axis=0), 1))
# # X = np.delete(X, obj=to_delete, axis=1)

# X = 0.5 * X

# Run mine

In [4]:
class SimpleClassifier(nn.Module):
    
    def __init__(self, in_size, out_size):
        super(SimpleClassifier, self).__init__()
        
        self.in_size = in_size
        self.out_size = out_size
        
        self.model = nn.Sequential()
        self.model.add_module(
            "linear", nn.Linear(in_size, out_size)
        )
        
    def forward(self, x):
        return self.model(x)
    
    def probs(self, x):
        scores = self.model(x).numpy()
        probs = np.exp(scores)
        probs /= probs.sum()
        return probs
    
    def train(self, X, y, num_epochs=20, batch_size=20):
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=1e-4)
        
        num_train = X.shape[0]
        num_batches = int(np.ceil(num_train / batch_size))
        
        stats = np.zeros(num_epochs)
        for epoch in num_epochs:
            shuf_idx = np.random.choice(num_train, size=num_train, replace=True)
            
            avg_loss = 0.
            for batch_num in range(num_batches):
                s = batch_num * batch_size
                e = s + batch_size
                idx = shuf_idx[s:e]
                X_batch = Variable(torch.Tensor(X[idx]))
                y_true = Variable(torch.Tensor(y[idx]))
                
                optimizer.zero_grad()
                
                y_pred = self(X_batch)
                loss = loss_function(y_pred, y_target)
                loss.backward()
                optimizer.step()
                
                avg_loss += loss.data
                
            avg_loss /= num_batches
            stats[epoch] = avg_loss
            
        return stats
    
pass

In [ ]:
num_epochs = 5
batch_size = 20
num_batches = int(np.ceil(num_data / batch_size))

num_visible = num_dims
num_hidden = 500


def train_rbm(
    rbm, data_loader, num_epochs, batch_size, jupyter=False, 
    outer_hooks=[], inner_hooks=[]
):
    if jupyter:
        progress = IntProgress(value=0, max=num_batches, description='Epo=0')
        display(progress)

    errors = []

    for epoch in range(num_epochs):

        if jupyter:
            progress.value = 0
            progress.description = 'Epo=%d' % epoch

        for b_num, (x, _) in enumerate(data_loader.load_batches(batch_size)):
            g, e = rbm.CD(x, n=1, lr=0.001)
            errors.append(e)
            
            if jupyter:
                progress.value += 1
                
            for hook in inner_hooks:
                hook(rbm, epoch, b_num)
                
        for hook in outer_hooks:
            hook(rbm, epoch, 0)

    if jupyter:
        progress.close()

    return rbm, errors


class ClassifierResults(object):
    
    def __init__(
        self, data_loader, batch_size, num_epochs, eval_freq=20, jupyter=False
    ):
        self.data_loader = data_loader
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.eval_freq = eval_freq
        self.jupyter = jupyter
        
        self.results = defaultdict(dict)
        
    
    def train_classifier(self, rbm, epoch, b_num):
        if (b_num % self.eval_freq) != 0:
            return
        
        in_size = rbm.H
        out_size = self.data_loader.num_targets
        
        classifier = SimpleClassifier(in_size, out_size)
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(classifier.parameters(), lr=1e-4)
        
        if self.jupyter:
            prog = IntProgress(
                value=0, max=self.num_epochs, description='Training classifier...'
            )
            display(prog)
        
        stats = np.zeros(self.num_epochs)        
        for epoch_clf in range(self.num_epochs):
            avg_loss = 0.
            
            for (X, y) in self.data_loader.load_batches(self.batch_size):
                X = rbm.transform(X)
                X_batch = Variable(torch.Tensor(X))
                
                y = np.argwhere(y == 1.)[:, 1]
                y_true = Variable(torch.LongTensor(y))
                
                optimizer.zero_grad()

                y_pred = classifier(X_batch)
                loss = loss_function(y_pred, y_true)
                loss.backward()
                optimizer.step()

                avg_loss += loss.data
        
            avg_loss /= num_batches
            stats[epoch_clf] = avg_loss
            
            if self.jupyter:
                prog.value += 1
                
        if self.jupyter:
            prog.close()
            
        self.results[epoch][b_num] = stats

classifier_results = ClassifierResults(
    data_loader, batch_size=20, num_epochs=20, eval_freq=20, jupyter=True
)
inner_hooks = [classifier_results.train_classifier]

rbm = RBM(num_hidden, num_visible)
rbm, errors = train_rbm(
    rbm, data_loader, num_epochs, batch_size, jupyter=True, inner_hooks=inner_hooks
)

# f, ax = plt.subplots(figsize=(12, 7))
# ax.plot(range(len(errors)), errors)
# mn, mx = ax.get_ylim()

# for ep in range(0, num_batches * num_epochs, num_batches):
#     ax.plot([ep, ep], [mn, mx], color='red', linestyle='--', alpha=0.4)

# ax.set_ylim([mn, mx])
# plt.show()

        


IntProgress(value=0, description='Epo=0', max=53)

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

...

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

IntProgress(value=0, description='Training classifier...', max=20)

In [ ]:
res = classifier_results.results

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 5))

xs = range(classifier_results.num_epochs)
ax1.plot(xs, list(res[19][0]))
ax2.plot(xs, list(res[19][20]))
ax3.plot(xs, list(res[19][40]))

plt.show()

# DRBM